# **Modelling and Evaluation C - Benign and Malignant classification of Skin Lesions**

## Objectives

Answer Business requirement 2:
The client wants to be able to know from the photo of a skin lesion if it is a bening or malignant one.

## Inputs
* input/Benign_Malignant_Dataset/MainData/  
    * test  
      * benign  
      * malignant  
    * train  
      * benign  
      * malignant   
    * validation  
      * benign  
      * malignant 
* image shape embeddings

## Outputs

* Images distribution plot in train, validation, and test set
* Image augmentation
* Class indices to change prediction inference in labels
* Machine learning model creation and training
* Saved model
* Learning curve plot for model performance
* Model evaluation on pickle file
* Prediction on the random image file.

---

### Change working directory

We need to change the working directory from its current folder to its parent folder

* We access the current directory with os.getcwd()

In [1]:
import os
current_dir = os.getcwd()
current_dir

'/Users/carolina/Documents/CodeInstitute/benign-malignant-classification/jupyter_notebooks'

We want to make the parent of the current directory the new current directory

* os.path.dirname() gets the parent directory
* os.chir() defines the new current directory


In [2]:
os.chdir(os.path.dirname(current_dir))
print("You set a new current directory")

You set a new current directory


Confirm the new current directory

In [3]:
working_dir = os.getcwd()
working_dir

'/Users/carolina/Documents/CodeInstitute/benign-malignant-classification'

### Import regular packages

In [4]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.image import imread

### Set input directories

Set train, validation and test paths

In [5]:
input_dir = 'input/Benign_Malignant_DataSet/MainData/'
train_path = os.path.join(input_dir, 'train')
test_path = os.path.join(input_dir, 'test')
validation_path = os.path.join(input_dir, 'validation')

In [6]:
version = 'v1'
file_path = os.path.join('outputs', version)

if 'outputs' in os.listdir(working_dir) and version in \
        os.listdir(os.path.join(working_dir, 'outputs')):
    print('This version of the output dataset already exists.')
    pass
else:
    os.makedirs(name=file_path)

This version of the output dataset already exists.


### Set label names

In [7]:
labels = os.listdir(train_path)
print(f'Project lables: {labels}')

Project lables: ['malignant', 'benign']


## Load image shape

### Import saved image shape embedding

In [ ]:
import joblib

image_shape = joblib.load(filename=f'outputs/{version}/image_shape.pkl')
image_shape